In [1]:
import yaml
from AntController.HyperparamSearchUtils import create_data_from_data_stores, \
            create_actor_func_from_hyper_param_search, \
            create_value_func_from_hyper_param_search, \
            pretrain_predictor_as_value_func, \
            get_value_func_loss

from AntController.AntEnvironment import EpisodeData
from AntController.HaikuPredictor import HaikuPredictor
import os
import numpy as np

from ServoController.WalktCycleConfigParser import WalkCycle

In [2]:
training_data_dir = "TrainingData/Fixed_Walk_With_Sensor"
data = create_data_from_data_stores(training_data_dir, False)


In [3]:
rewards, states, actions, shifted_states, is_non_terminal = data
discount = 0.98
test_pivot = int(0.1 * len(data[0]))

In [4]:
non_terminal_indices = np.where(is_non_terminal >0.5)[0]
orientation_index =  [10, 11]
pos_index =[8, 9]
robot_index = range(8)
action_index = [0,4]

In [5]:
states[non_terminal_indices][:, orientation_index]

array([[0.9939087 , 0.11020683],
       [0.99639404, 0.0848466 ],
       [0.99906456, 0.04324287],
       ...,
       [0.97543067, 0.22030647],
       [0.97444797, 0.2246134 ],
       [0.97716719, 0.21247165]])

In [6]:
states[non_terminal_indices][:, pos_index]

array([[0.06658623, 0.38680655],
       [0.06685618, 0.38806674],
       [0.06992043, 0.38879454],
       ...,
       [0.77231675, 0.42561141],
       [0.77554476, 0.42993438],
       [0.78005993, 0.43688837]])

In [7]:
velocity = (shifted_states[:, pos_index] - states[:, pos_index] )*100

In [8]:
orientation = states[:, orientation_index]

In [9]:
velocity[non_terminal_indices]

array([[ 0.0269942 ,  0.12601912],
       [ 0.30642524,  0.07278025],
       [ 0.29643551, -0.24676919],
       ...,
       [ 0.32280087,  0.43229759],
       [ 0.45151711,  0.69539845],
       [ 0.90805888,  0.43273568]])

In [10]:
velocity_reward = np.sum(np.multiply(velocity[non_terminal_indices], orientation[non_terminal_indices]), axis =1)
velocity_reward = velocity_reward.reshape((len(velocity_reward), 1))

In [11]:
input_dataset = WalkCycle(
    "WalkConfigs/nn_training_walk_config.yaml", speed=0.3
).get_all_frames()

In [12]:
frames = np.asarray(input_dataset)

In [13]:
frames

array([[ 0.15 , -0.15 ,  0.15 , -0.15 ,  1.   ,  0.7  ,  1.   ,  0.7  ],
       [ 0.075, -0.075,  0.075, -0.075,  1.   ,  0.7  ,  1.   ,  0.7  ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.7  ,  1.   ,  0.7  ],
       [-0.075,  0.075, -0.075,  0.075,  1.   ,  0.7  ,  1.   ,  0.7  ],
       [-0.15 ,  0.15 , -0.15 ,  0.15 ,  1.   ,  0.7  ,  1.   ,  0.7  ],
       [-0.15 ,  0.15 , -0.15 ,  0.15 ,  0.7  ,  1.   ,  0.7  ,  1.   ],
       [-0.075,  0.075, -0.075,  0.075,  0.7  ,  1.   ,  0.7  ,  1.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.7  ,  1.   ,  0.7  ,  1.   ],
       [ 0.075, -0.075,  0.075, -0.075,  0.7  ,  1.   ,  0.7  ,  1.   ],
       [ 0.15 , -0.15 ,  0.15 , -0.15 ,  0.7  ,  1.   ,  0.7  ,  1.   ]])

In [14]:
def find_closest_pos_index(robot_state):
    distance = np.linalg.norm(frames - robot_state, axis =1)
    return np.argmin(distance)

In [15]:
phase_state = np.asarray([find_closest_pos_index(robot_state) for robot_state in states[:,robot_index]])

In [16]:
next_phase_state = np.asarray([find_closest_pos_index(robot_state) for robot_state in shifted_states[:,robot_index]])

In [17]:
def to_phase_vector(index, steps):
    angle = 2*np.pi*index/steps
    return np.dstack((np.cos(angle), np.sin(angle)))[0]

In [20]:
phase_vector = to_phase_vector(phase_state, 10)[non_terminal_indices]
next_phase_vector = to_phase_vector(next_phase_state, 10)[non_terminal_indices]

In [21]:
phase_vector = np.concatenate((phase_vector, to_phase_vector(phase_state+5, 10)[non_terminal_indices])) 
next_phase_vector = np.concatenate((next_phase_vector, to_phase_vector(next_phase_state+5, 10)[non_terminal_indices]))
velocity_reward = np.concatenate((velocity_reward,velocity_reward))

In [22]:
batch_size = 256

In [23]:
def train_on_batch(predictor, discount):
    l=len(phase_vector)
    batch_ids = np.random.choice(l ,batch_size)
    inputs = phase_vector[batch_ids]
    rewards = velocity_reward[batch_ids] + predictor.evaluate(next_phase_vector[batch_ids])*discount
    return predictor.train_batch(inputs,rewards)/batch_size

In [28]:
path = "AntController/configs/fixed_cycle_critic_trainer_config.yaml"
with open(path) as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
walk_critic = HaikuPredictor.generate_controller_from_config(config)

In [32]:
for i in range(1000):
    train_on_batch(walk_critic, 0.95)
train_on_batch(walk_critic, 0.95)

DeviceArray(0.2558332, dtype=float32)

In [33]:
batch_ids = np.random.choice(len(phase_vector), 5)

In [34]:
walk_critic.evaluate(next_phase_vector[:11])

DeviceArray([[17.613783],
             [17.6103  ],
             [17.371634],
             [17.095972],
             [17.466585],
             [17.838734],
             [17.841269],
             [17.508429],
             [17.095846],
             [17.328321],
             [17.613783]], dtype=float32)

In [36]:
import pickle

save_dir = "AntController/configs/"

def save_models_with_config(path, config, model):
    saved_params = (config, model.params, model.optimizer_state)
    pickle.dump(saved_params, open( path, "wb" ) )
    
def get_model_with_config(path):
    config, params, optimizer_state = pickle.load(open(path, "rb" ))
    predictor = HaikuPredictor.generate_controller_from_config(config)
    predictor.params = params
    predictor.optimizer_state = optimizer_state
    return predictor, config


In [38]:
save_path = os.path.join(save_dir, "fixed_cycle_critic.p")
save_models_with_config(save_path, config, walk_critic)

In [4]:
def get_phase_vector(length):
    angle = 2 * np.pi * np.arange(length) / length
    return np.dstack((np.cos(angle), np.sin(angle)))[0]


def full_action_to_quarter_action(action):
    return action[:, [0, 4]]


def get_vector_from_angle(angle):
    return np.dstack((np.cos(angle), np.sin(angle)))



path = "AntController/configs/fixed_cycle_trainer_config.yaml"
with open(path) as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
actor = HaikuPredictor.generate_controller_from_config(config)
input_dataset = WalkCycle(
    "WalkConfigs/nn_training_walk_config.yaml", speed=0.3
).get_all_frames()
actions = full_action_to_quarter_action(np.asarray(input_dataset)) 
states = get_phase_vector(len(actions))

print(len(actions), len(states))

for _ in range(100):
    loss = actor.train_batch(states, actions)
    print(loss)


10 10
10.779549
10.211185
9.668233
9.151503
8.660837
8.195132
7.752788
7.332155
6.9318714
6.550897
6.1884065
5.84367
5.515998
5.2047176
4.9091673
4.628695
4.362657
4.1104145
3.8713417
3.6448174
3.4302292
3.2269745
3.0344584
2.8521018
2.679338
2.5156152
2.3604014
2.2131732
2.0734682
1.9408551
1.8149184
1.6952872
1.5816314
1.4736633
1.3711307
1.2738175
1.1815372
1.0941341
1.0114707
0.93342245
0.85987806
0.79072297
0.72584784
0.6651457
0.6084887
0.55576384
0.50685
0.46161988
0.4199339
0.3816465
0.3466062
0.31465468
0.28562802
0.25935796
0.23567057
0.21438938
0.19533479
0.17832793
0.16318986
0.14974405
0.13781916
0.12724958
0.117877856
0.10955648
0.10214946
0.09553623
0.089651674
0.08497588
0.082068354
0.075847715
0.071230225
0.0678083
0.064037584
0.0609771
0.058088966
0.055389505
0.052963443
0.050742973
0.04870833
0.046863362
0.045200758
0.04371006
0.04238021
0.041200757
0.040160805
0.039248317
0.038451716
0.037761137
0.03717905
0.036828175
0.037604727
0.038395416
0.035792902
0.035447128


In [9]:
save_path = os.path.join(save_dir, "fixed_cycle_actor.p")
save_models_with_config(save_path, config, actor)